In [5]:
import spotipy
from dotenv import load_dotenv
import pandas as pd
import numpy as np

In [4]:
load_dotenv()

True

In [9]:
    scope = ['ugc-image-upload'
            ,'user-read-recently-played'
            ,'user-top-read'
            ,'user-read-playback-position'
            ,'user-read-playback-state'
            ,'user-modify-playback-state'
            ,'user-read-currently-playing'
            ,'app-remote-control'
            ,'streaming'
            ,'playlist-modify-public'
            ,'playlist-modify-private'
            ,'playlist-read-private'
            ,'playlist-read-collaborative'
            ,'user-follow-modify'
            ,'user-follow-read'
            ,'user-library-modify'
            ,'user-library-read'
            ,'user-read-email'
            ,'user-read-private'
            ]


    auth_manager = spotipy.oauth2.SpotifyOAuth(scope=scope,
                                                show_dialog=True)

    # auth_manager.get_access_token(request.args.get("code"))
    auth_url = auth_manager.get_authorize_url()
    spotify = spotipy.Spotify(auth_manager=auth_manager)


In [115]:
artists_items = spotify.current_user_top_artists(limit=100, time_range='long_term')['items']

In [116]:
temp = pd.DataFrame.from_dict(artists_items)

In [113]:
def clean_top_artists(items_df):
    '''
        
    '''

    items_df['external_urls'] = temp['external_urls'].map(lambda x: x['spotify'])
    items_df['images'] = temp['images'].map(lambda x: x[0]['url'])
    items_df['followers'] = temp['followers'].map(lambda x: x['total'])

    return items_df

In [110]:
temp['external_urls'] = temp['external_urls'].map(lambda x: x['spotify'])
temp['images'] = temp['images'].map(lambda x: x[0]['url'])
temp['followers'] = temp['followers'].map(lambda x: x['total'])
temp.to_csv('top_artists_long.csv')

In [121]:
spotify.current_user_top_tracks()['items'][0]['album']

{'album_type': 'ALBUM',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7nQKsOwNp5gENU4Ckw9ddD'},
   'href': 'https://api.spotify.com/v1/artists/7nQKsOwNp5gENU4Ckw9ddD',
   'id': '7nQKsOwNp5gENU4Ckw9ddD',
   'name': 'CT Fletcher',
   'type': 'artist',
   'uri': 'spotify:artist:7nQKsOwNp5gENU4Ckw9ddD'}],
 'available_markets': ['AD',
  'AE',
  'AR',
  'AT',
  'AU',
  'BE',
  'BG',
  'BH',
  'BO',
  'BR',
  'CA',
  'CH',
  'CL',
  'CO',
  'CR',
  'CY',
  'CZ',
  'DE',
  'DK',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FR',
  'GB',
  'GR',
  'GT',
  'HK',
  'HN',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IS',
  'IT',
  'JO',
  'JP',
  'KW',
  'LB',
  'LI',
  'LT',
  'LU',
  'LV',
  'MA',
  'MC',
  'MT',
  'MX',
  'MY',
  'NI',
  'NL',
  'NO',
  'NZ',
  'OM',
  'PA',
  'PE',
  'PH',
  'PL',
  'PT',
  'PY',
  'QA',
  'RO',
  'SA',
  'SE',
  'SG',
  'SK',
  'SV',
  'TH',
  'TN',
  'TR',
  'TW',
  'US',
  'UY',
  'VN',
  'ZA'],
 'external_urls': {'s

In [123]:
spotify.current_user_top_tracks()['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/7nQKsOwNp5gENU4Ckw9ddD'},
  'href': 'https://api.spotify.com/v1/artists/7nQKsOwNp5gENU4Ckw9ddD',
  'id': '7nQKsOwNp5gENU4Ckw9ddD',
  'name': 'CT Fletcher',
  'type': 'artist',
  'uri': 'spotify:artist:7nQKsOwNp5gENU4Ckw9ddD'}]

In [124]:
spotify.current_user_top_tracks()['items'][0].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [125]:
track_items =spotify.current_user_top_tracks()['items']

In [128]:
temp = pd.DataFrame.from_dict(track_items)

In [139]:
temp.head(6)

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,166896,True,{'isrc': 'US3DF1602912'},{'spotify': 'https://open.spotify.com/track/33...,https://api.spotify.com/v1/tracks/334ddOVtCHm6...,334ddOVtCHm6FlifbA6bKF,False,F*Ck Average,37,https://p.scdn.co/mp3-preview/7da7f1db6ee742e4...,2,track,spotify:track:334ddOVtCHm6FlifbA6bKF
1,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,306431,False,{'isrc': 'QMQTM1500473'},{'spotify': 'https://open.spotify.com/track/0H...,https://api.spotify.com/v1/tracks/0HBYptBUJX7N...,0HBYptBUJX7NBm82ogaw1y,False,Beast Mode,40,https://p.scdn.co/mp3-preview/059fbfbf5230bb7a...,6,track,spotify:track:0HBYptBUJX7NBm82ogaw1y
2,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,120480,True,{'isrc': 'US3DF1602911'},{'spotify': 'https://open.spotify.com/track/0P...,https://api.spotify.com/v1/tracks/0PSWclszGNAB...,0PSWclszGNABEONvVZ3R9o,False,Pre-Workout,36,https://p.scdn.co/mp3-preview/cd9b53c289f6bed1...,1,track,spotify:track:0PSWclszGNABEONvVZ3R9o
3,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",2,159512,False,{'isrc': 'USUG11600374'},{'spotify': 'https://open.spotify.com/track/0Z...,https://api.spotify.com/v1/tracks/0ZAiqymvbQUZ...,0ZAiqymvbQUZAh9y6ZT3Rm,False,XO,72,None,2,track,spotify:track:0ZAiqymvbQUZAh9y6ZT3Rm
4,"{'album_type': 'SINGLE', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,118571,False,{'isrc': 'QM24S2100336'},{'spotify': 'https://open.spotify.com/track/5Q...,https://api.spotify.com/v1/tracks/5QGHOeBYDauP...,5QGHOeBYDauPtC8yYD3wZZ,False,Parallel Parking,64,https://p.scdn.co/mp3-preview/b94bdb35917c625b...,1,track,spotify:track:5QGHOeBYDauPtC8yYD3wZZ
5,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,180053,False,{'isrc': 'USSM12002051'},{'spotify': 'https://open.spotify.com/track/4C...,https://api.spotify.com/v1/tracks/4CxmynXhw78Q...,4CxmynXhw78QefruycvxG8,False,Coffee (feat. Marc E. Bassy),69,https://p.scdn.co/mp3-preview/5a2101969adeb008...,5,track,spotify:track:4CxmynXhw78QefruycvxG8


In [137]:
temp['album'][5]

{'album_type': 'ALBUM',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3ApUX1o6oSz321MMECyIYd'},
   'href': 'https://api.spotify.com/v1/artists/3ApUX1o6oSz321MMECyIYd',
   'id': '3ApUX1o6oSz321MMECyIYd',
   'name': 'Quinn XCII',
   'type': 'artist',
   'uri': 'spotify:artist:3ApUX1o6oSz321MMECyIYd'}],
 'available_markets': ['AD',
  'AE',
  'AR',
  'AT',
  'AU',
  'BE',
  'BG',
  'BH',
  'BO',
  'BR',
  'CA',
  'CH',
  'CL',
  'CO',
  'CR',
  'CY',
  'CZ',
  'DE',
  'DK',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FR',
  'GB',
  'GR',
  'GT',
  'HK',
  'HN',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IS',
  'IT',
  'JO',
  'JP',
  'KW',
  'LB',
  'LI',
  'LT',
  'LU',
  'LV',
  'MA',
  'MC',
  'MT',
  'MX',
  'MY',
  'NI',
  'NL',
  'NO',
  'NZ',
  'OM',
  'PA',
  'PE',
  'PH',
  'PL',
  'PS',
  'PT',
  'PY',
  'QA',
  'RO',
  'SA',
  'SE',
  'SG',
  'SK',
  'SV',
  'TH',
  'TN',
  'TR',
  'TW',
  'US',
  'UY',
  'VN',
  'ZA'],
 'external_url

In [138]:
temp['artists'][5]

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/3ApUX1o6oSz321MMECyIYd'},
  'href': 'https://api.spotify.com/v1/artists/3ApUX1o6oSz321MMECyIYd',
  'id': '3ApUX1o6oSz321MMECyIYd',
  'name': 'Quinn XCII',
  'type': 'artist',
  'uri': 'spotify:artist:3ApUX1o6oSz321MMECyIYd'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/3tQx1LPXbsYjE9VwN1Peaa'},
  'href': 'https://api.spotify.com/v1/artists/3tQx1LPXbsYjE9VwN1Peaa',
  'id': '3tQx1LPXbsYjE9VwN1Peaa',
  'name': 'Marc E. Bassy',
  'type': 'artist',
  'uri': 'spotify:artist:3tQx1LPXbsYjE9VwN1Peaa'}]